In [ ]:
import os
import torch
import torch.nn as nn

In [ ]:
model_tcga = "/lustre/groups/shared/users/peng_marr/HistoDINO/logs/DINOv2_finetuned/dinov2_vits_TCGA_training_29999_teacher_checkpoint.pth"
model_nct = "/lustre/groups/shared/users/peng_marr/HistoDINO/logs/DINOv2_finetuned/dinov2_vits_NCT_10k_training_1999_teacher_checkpoint.pth"
model_tcga_brca = "/lustre/groups/shared/users/peng_marr/HistoDINO/vits14_brca/training_29999/teacher_checkpoint.pth"
model_tcga_nsclc = "/lustre/groups/shared/users/peng_marr/HistoDINO/vits14_nsclc/training_29999/teacher_checkpoint.pth"
model_paths = [
    # model_tcga, 
    # model_nct, 
    model_tcga_brca, 
    model_tcga_nsclc
]

In [ ]:
NUM_MODELS = len(model_paths)

In [ ]:
torch.load(model_tcga)

In [ ]:
# create the uniform soup sequentially to not overload memory
for j, model_path in enumerate(model_paths):

    print(f'Adding model {j} of {NUM_MODELS - 1} to uniform soup.')

    assert os.path.exists(model_path)
    state_dict = torch.load(model_path)['teacher']
    if j == 0:
        uniform_soup = {k : v * (1./NUM_MODELS) for k, v in state_dict.items()}
    else:
        uniform_soup = {k : v * (1./NUM_MODELS) + uniform_soup[k] for k, v in state_dict.items()}

In [ ]:
state_dict.keys()

In [ ]:
ckpt = {'teacher': uniform_soup}

In [ ]:
save_path = "/lustre/groups/shared/users/peng_marr/HistoDINO/logs/merged_models/dinov2_finetuned_vits14_TCGA-BRCA-NSCLC_uniform_soup.pth"
torch.save(ckpt, save_path)

In [ ]:
def get_dino_finetuned_downloaded(model_path,modelname):
    # pos_embed has wrong shape
    model = torch.hub.load("facebookresearch/dinov2", modelname)
    # model=torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14')
    # load finetuned weights
    if model_path is not None:
        pretrained = torch.load(model_path, map_location=torch.device("cpu"))
        # make correct state dict for loading
        new_state_dict = {}
        for key, value in pretrained["teacher"].items():
            if "dino_head" in key or "ibot_head" in key:
                pass
            else:
                new_key = key.replace("backbone.", "")
                new_state_dict[new_key] = value
        # change shape of pos_embed
        pos_embed = nn.Parameter(torch.zeros(1, 257, 384))
        # pos_embed = nn.Parameter(torch.zeros(1, 257, 1536))
        model.pos_embed = pos_embed
        # load state dict
        model.load_state_dict(new_state_dict, strict=True)
    return model

In [ ]:
path = "/lustre/groups/shared/users/peng_marr/HistoDINO/logs/merged_models/dinov2_finetuned_vits_TCGA_NCT_uniform_soup.pth"

In [ ]:
model = get_dino_finetuned_downloaded(save_path, "dinov2_vits14")

In [ ]:
input = torch.randn(1, 3, 224, 224).cuda()
model = model.cuda()
with torch.no_grad():
    out = model(input)
print(out.shape)